In [5]:
import torch
from torch import nn

# _____________________________________________________________________________________________________________________
# <center>**Analysis and Comparison**</center>
# 
# 
# 

##  **MLP**

#### Haha we changed the code for model architecture to go from experiment 1 to experiment 2 and the architecture for experiment 1 is not saved anywhere; the weights are though. For reconstructing the model for experiment 1, I will proceed by guessing it from layer dimensions.

### **Experiment 1**:

In [6]:
checkpoints_1 = torch.load("./models/checkpoint_Exp1.pth")

for key in checkpoints_1['model_state_dict'].keys():
    print(key, checkpoints_1['model_state_dict'][key].shape)

layers.0.weight torch.Size([1024, 3072])
layers.0.bias torch.Size([1024])
layers.2.weight torch.Size([10, 1024])
layers.2.bias torch.Size([10])


#### Ok clear now. And layer 1 is ReLu so no weights and biases there. For experiment 1, I will have to reconstruct its MLP class.

In [7]:
state_dict = checkpoints_1['model_state_dict']

In [8]:
class MLP1(nn.Module):
    """
    MLP composed of two fully connected layers.
     - First layer takes pixel values and maps them to a hidden dimension
     - Nonlinear activation
     - Second layer maps from hidden dimension to number of classes, predicting a score for each of the classes
    """
    def __init__(self, input_dim=3072, hidden_dim=1024, output_dim=10):
        """ Model initalizer """
        super().__init__()
        self.layers = nn.Sequential(
                nn.Linear(in_features=input_dim, out_features=hidden_dim),
                nn.ReLU(),
                nn.Linear(in_features=hidden_dim, out_features=output_dim)
            )
        
    def forward(self, x):
        """ Forward pass through the model"""
        assert len(x.shape) == 2, f"ERROR! Shape of input must be 2D (b_size, dim)"
        pred = self.layers(x)
        return pred

In [9]:
model_1 = MLP1()
model_1.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

### **Experiment 2:**

## **CONV**